# This ipynb first generate MLP-like QBAFs and then compute the desired ordering.

### Step1: Generate random MLP-like QBAFs

In [19]:
import random
random.seed(1)
import sys
sys.path.append("../../src/")
import uncertainpy.gradual as grad
import numpy as np
from tqdm import tqdm

In [20]:
def generate_random_mlp_graph(layer_sizes, connection_prob):
    """
    generate a random MLP-like QBAF structure, represented by DAG

    parameter:
    - connection_prob: The probability of inter-layer connections (1.0 = fully connected, 0.0 = no connections)

    return:
    - graph: 邻接表表示的 MLP 结构 (DAG)
    """

    graph = {}  # Store the DAG using an adjacency list
    node_id = 0  # neuron ID
    layer_nodes = []  # Record the neuron IDs in each layer

    # Create neuron nodes
    for size in layer_sizes:
        layer = [node_id + i for i in range(size)]
        layer_nodes.append(layer)
        node_id += size

    # Generate inter-layer connections
    for i in range(len(layer_nodes) - 1):  # Layer-by-layer connection
        for src in layer_nodes[i]:  # Current layer neurons
            for dst in layer_nodes[i+1]:  # Next layer neurons
                if random.uniform(0, 1) < connection_prob:
                    graph.setdefault(src, set()).add(dst)

    # if node not in graph, add empty set
    for layer in layer_nodes:
        for node in layer:
            graph.setdefault(node, set())

    return graph

In [21]:
def get_layer_nodes(layer_sizes, layer_index=None):
    node_id = 0
    for i, size in enumerate(layer_sizes):
        if i == layer_index:
            return [str(n) for n in range(node_id, node_id + size)]
        node_id += size

    raise ValueError(f"Invalid layer_index {layer_index}: must be between 0 and {len(layer_sizes) - 1}")

# preferred_order = get_layer_nodes(layer_sizes, len(layer_sizes)-1)

In [22]:
# generate a random MLP-QBAF and output to a file
def generate_and_write_graph(filename, layer_sizes, connection_prob):
    with open(filename, 'w') as f:
        sys.stdout = f

        # generate the node and edge of a graph
        random_graph = generate_random_mlp_graph(layer_sizes, connection_prob)

        L0 = get_layer_nodes(layer_sizes, len(layer_sizes)-1)
        L1 = get_layer_nodes(layer_sizes, len(layer_sizes)-2)
        L2 = get_layer_nodes(layer_sizes, len(layer_sizes)-3)
        L3 = get_layer_nodes(layer_sizes, len(layer_sizes)-4)

        L0 = list(map(int, L0))
        L1 = list(map(int, L1))
        L2 = list(map(int, L2))
        L3 = list(map(int, L3))

        # generate random base scores for arguments
        for node, edges in random_graph.items():
            # print(node)
            if node in L1:
                random_float = round(random.uniform(0.0, 0.1),2)
            else:
                random_float = round(random.uniform(0.0, 1.0),2)
            print(f"arg({node}, {random_float}).")

        # generate random polarity for edges
        for node, edges in random_graph.items():
            for edge in edges:

                if edge in L1:
                    random_boolean = True
                elif edge in L2:
                    random_boolean = False
                else:
                    random_boolean = random.choice([True, False])
                if random_boolean:
                    print(f"att({node}, {edge}).")
                else:
                    print(f"sup({node}, {edge}).")

    sys.stdout = sys.__stdout__

In [23]:
# set generation parameters
connection_prob=1.0
layer_sizes = [8,8,8,3]
mlp_graph = generate_random_mlp_graph(layer_sizes, connection_prob)

print("MLP structure:", layer_sizes)

In [24]:
mlp_graph

{0: {8, 9, 10, 11, 12, 13, 14, 15},
 1: {8, 9, 10, 11, 12, 13, 14, 15},
 2: {8, 9, 10, 11, 12, 13, 14, 15},
 3: {8, 9, 10, 11, 12, 13, 14, 15},
 4: {8, 9, 10, 11, 12, 13, 14, 15},
 5: {8, 9, 10, 11, 12, 13, 14, 15},
 6: {8, 9, 10, 11, 12, 13, 14, 15},
 7: {8, 9, 10, 11, 12, 13, 14, 15},
 8: {16, 17, 18, 19, 20, 21, 22, 23},
 9: {16, 17, 18, 19, 20, 21, 22, 23},
 10: {16, 17, 18, 19, 20, 21, 22, 23},
 11: {16, 17, 18, 19, 20, 21, 22, 23},
 12: {16, 17, 18, 19, 20, 21, 22, 23},
 13: {16, 17, 18, 19, 20, 21, 22, 23},
 14: {16, 17, 18, 19, 20, 21, 22, 23},
 15: {16, 17, 18, 19, 20, 21, 22, 23},
 16: {24, 25, 26},
 17: {24, 25, 26},
 18: {24, 25, 26},
 19: {24, 25, 26},
 20: {24, 25, 26},
 21: {24, 25, 26},
 22: {24, 25, 26},
 23: {24, 25, 26},
 24: set(),
 25: set(),
 26: set()}

In [25]:
N = 100 # generate N QBAFs storing in N files
for i in range(N):
    filename = f'../../bags/mlp_{i}.bag'
    generate_and_write_graph(filename, layer_sizes, connection_prob)

### Step 2: computer desired orderings

In [26]:
# obtain a QBAF and set the gradual semantics

# Computation via Forward Propagation
bag = grad.BAG("../../bags/mlp_0.bag")

# # QE
# agg_f = grad.semantics.modular.SumAggregation()
# inf_f = grad.semantics.modular.QuadraticMaximumInfluence(conservativeness=1)

# # DF-QuAD
# agg_f = grad.semantics.modular.ProductAggregation()
# inf_f = grad.semantics.modular.LinearInfluence(conservativeness=1)

# # SD-DF-QUAD
# agg_f = grad.semantics.modular.ProductAggregation()
# inf_f = grad.semantics.modular.SQDFQUADInfluence(conservativeness=1)

# EB
agg_f = grad.semantics.modular.SumAggregation()
inf_f = grad.semantics.modular.EulerBasedInfluence()

# # EBT
# agg_f = grad.semantics.modular.TopAggregation()
# inf_f = grad.semantics.modular.EulerBasedInfluence()

#returns dictionary of strength values if needed
strength_values = grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)

for arg in bag.arguments.values():
    print((arg.name,arg.strength))

In [27]:
def get_layer_nodes(layer_sizes, layer_index=None):
    node_id = 0
    for i, size in enumerate(layer_sizes):
        if i == layer_index:
            return [str(n) for n in range(node_id, node_id + size)]
        node_id += size

    raise ValueError(f"Invalid layer_index {layer_index}: must be between 0 and {len(layer_sizes) - 1}")

In [28]:
preferred_order = get_layer_nodes(layer_sizes, len(layer_sizes)-1)
preferred_order

['24', '25', '26']

In [29]:
immutable_args = get_layer_nodes(layer_sizes, len(layer_sizes)-2)
immutable_args

['16', '17', '18', '19', '20', '21', '22', '23']

In [30]:
# # simple ReLU loss function (linear)
# def compute_loss(bag, preferred_order):
#     loss = 0
#     for i in range(len(preferred_order) - 1):
#         loss += max(0, (bag.arguments[preferred_order[i+1]].strength - bag.arguments[preferred_order[i]].strength))
#     # print(f"Loss: {loss}")
#     return loss

In [31]:
# pairwise smooth logistic loss (quadratic)
def compute_loss(bag, preferred_order,delta=0):
    loss = 0
    # traverse all the pairs in preferred_order, i < j
    for i in range(len(preferred_order) - 1):
        for j in range(i + 1, len(preferred_order)):
            A_i = preferred_order[i]
            A_j = preferred_order[j]
            # obtain strength σ(A_i) 和 σ(A_j)
            sigma_i = bag.arguments[A_i].strength
            sigma_j = bag.arguments[A_j].strength
            loss += np.log(1 + np.exp(sigma_j - sigma_i + delta))
    return loss

In [32]:
# compute gradient for the loss function
def compute_gradient(h, bag, preferred_order):
    # h is the perturbation score
    gradient = {}
    penalty = compute_loss(bag, preferred_order)
    for arg in bag.arguments.values():

        initial_weight = arg.get_initial_weight() # record initial base score
        arg.reset_initial_weight(initial_weight+h) # perturb base scores
        grad.algorithms.computeStrengthValues(bag, agg_f, inf_f) # recompute strength with new base score
        # for arg in bag.arguments.values():
        #     print((arg.name,arg.strength))

        new_penalty = compute_loss(bag, preferred_order)
        # print(new_penalty)

        gradient[arg.name] = (new_penalty-penalty)/h # compute gradient
        # print(f"gradient for argument {arg.name} is {gradient[arg.name]}")

        arg.reset_initial_weight(initial_weight) # set it back to the original base score after computing gradient
        grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)
        # for arg in bag.arguments.values():
        #     print((arg.name,arg.strength))

    return gradient

In [33]:
h = 10e-6
compute_gradient(h, bag, preferred_order)

{'0': -7.260414491838673e-07,
 '1': -7.260414491838673e-07,
 '2': -7.260414491838673e-07,
 '3': -7.260414491838673e-07,
 '4': -7.260414491838673e-07,
 '5': -7.260414491838673e-07,
 '6': -7.260414491838673e-07,
 '7': -7.260414491838673e-07,
 '8': -6.176392730594671e-07,
 '9': -1.4215295607300504e-07,
 '10': -1.3509193763638905e-07,
 '11': -6.920686246303375e-07,
 '12': -5.852651696613975e-07,
 '13': -2.6063595726100175e-07,
 '14': -2.3434587603787802e-07,
 '15': -2.2311041902867143e-07,
 '16': 0.0133789253009553,
 '17': 1.8582024807756168e-06,
 '18': 0.0133789253009553,
 '19': -0.0008423540087676428,
 '20': 0.000580033265862312,
 '21': 0.0008423539643587218,
 '22': 0.007497510612353152,
 '23': 0.000580033265862312,
 '24': -1.170547589834925,
 '25': 0.19770420425224697,
 '26': 0.9759826503419332}

In [34]:
# adam optimiser
def adam_gradient(name, gradient, m, v, i):

    grad = gradient[name]
    # Adam optimiser parameters
    learning_rate = 0.01  # Initial learning rate
    beta1 = 0.9            # First-order moment decay rate
    beta2 = 0.999          # Second-order moment decay rate
    epsilon = 1e-8         # a small constant

    # update first-order moment and second-order moment
    m = beta1 * m + (1 - beta1) * grad
    v = beta2 * v + (1 - beta2) * (grad ** 2)

    # bias correction
    m_hat = m / (1 - beta1 ** i)
    v_hat = v / (1 - beta2 ** i)

    update = learning_rate * m_hat / (np.sqrt(v_hat) + epsilon)

    return update, m, v

In [35]:
# if the order is exactly the same as the desired order, then valid otherwise not.
def is_valid_order(bag, preferred_order):
    strengths = [bag.arguments[name].strength for name in preferred_order]
    print(strengths)
    return all(strengths[i] > strengths[i+1] for i in range(len(strengths)-1))
print(is_valid_order(bag, preferred_order))

In [36]:
# test the validity for N MLP-like QBAFs
def main():
    valid = [0] * N
    for i in tqdm(range(N)):
        filename = f'../../bags/mlp_{i}.bag'
        bag = grad.BAG(filename)
        m = {}
        v = {}
        for epoch in range(1, 101):


            # print(f"Epoch:{epoch}")
            # compute gradient for all arguments
            gradient = compute_gradient(10e-6, bag, preferred_order)
            # print(f"gradient:{gradient}")
            if all(value == 0 for value in gradient.values()): break

            # update Adam state and update base scores
            for arg in bag.arguments.values():
                if arg.name not in immutable_args:
                    if arg.name not in m:
                        m[arg.name] = 0
                        v[arg.name] = 0


                    current_weight = arg.get_initial_weight()
                    adam_update, m[arg.name], v[arg.name] = adam_gradient(arg.name, gradient, m[arg.name], v[arg.name], epoch)
                    new_weight = current_weight - adam_update
                    new_weight = max(0, min(1, new_weight))
                    arg.reset_initial_weight(new_weight)


            # recompute the strength and penalty
            grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)
        if is_valid_order(bag, preferred_order):
            valid[i] = 1
    print(f"valid:{valid}")
if __name__ == "__main__":
    main()

100%|██████████| 100/100 [01:32<00:00,  1.08it/s]
